# **# Imports**




In [ ]:
!pip install ultralytics
!pip install torch

In [ ]:
# Imports
import numpy as np
import random
import torch
import shutil
import cv2
import os
from pathlib import Path
from ultralytics import YOLO
import torch
import subprocess

# **# Training YOLOv**
Este notebook muestra los pasos para entrenar un modelo YOLOvXX utilizando un conjunto de datos personalizado.

## Instalación de Dependencias

Antes de comenzar el entrenamiento, asegúrate de tener todas las dependencias necesarias instaladas.

> Esta es la estructura que necesitamos en nuestro DATASET para poder entrenar con YOLO
```
/images
    /train
    /val
/labels
    /train
    /val
dataset.yaml
```

> Esta es la estructura del archivo .yaml
```
train: ruta/a/images/train
val: ruta/a/images/val
nc: 2  # Número de clases
names: ['clase1', 'clase2'...]  # Nombres de las clases
```




In [ ]:
!nvidia-smi

In [ ]:
# Load YOLOv10s model from scratch
model = YOLO("yolov10m.pt")

# Train the model
model.train(data="/content/drive/MyDrive/Colab Notebooks/cs2-yolov10/datasets/headshot/data.yaml", epochs=100, imgsz=640)



# **# Tratamiento del DATASET**
## Procesamiento de Imágenes Usando OpenCV

En este cuaderno de Jupyter, vamos a aplicar diversas transformaciones a imágenes almacenadas en los directorios `train`, `valid` y `test`, que contienen las imágenes de entrenamiento y validación, respectivamente. Utilizaremos OpenCV para realizar operaciones como rotación, ajuste de brillo, contraste, escalado, y algunas transformaciones más extremas.

Además, moveremos las imágenes resultantes a directorios separados para una mejor organización.

> **Funciones de Transformación de Imágenes:**
- **Rotación**: Rotamos la imagen a diferentes ángulos (35, 65, 90, 180, 270 grados).
- **Brillo**: Ajustamos el brillo de la imagen, tanto aumentándolo como reduciéndolo.
- **Contraste**: Modificamos el contraste de la imagen para hacerla más o menos contrastada.
- **Escalado**: Escalamos la imagen, aumentándola al 150% y al 250%.

Estas transformaciones nos permitirán generar un conjunto más diverso de imágenes.






In [ ]:
# Función para rotar la imagen
def rotar_imagen(imagen, angulo):
    (alto, ancho) = imagen.shape[:2]
    centro = (ancho // 2, alto // 2)
    matriz_rotacion = cv2.getRotationMatrix2D(centro, angulo, 1.0)
    imagen_rotada = cv2.warpAffine(imagen, matriz_rotacion, (ancho, alto))
    return imagen_rotada

# Función para ajustar el brillo de la imagen
def ajustar_brillo(imagen, valor_brillo=30):
    hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)  # Convertir la imagen a HSV
    h, s, v = cv2.split(hsv)
    # Aumentar o disminuir el valor del brillo
    v = cv2.add(v, valor_brillo)
    v[v > 255] = 255  # Limitar el valor máximo
    v[v < 0] = 0  # Limitar el valor mínimo
    imagen_brillante = cv2.merge((h, s, v))
    imagen_brillante = cv2.cvtColor(imagen_brillante, cv2.COLOR_HSV2BGR)
    return imagen_brillante

# Función para ajustar el contraste de la imagen
def ajustar_contraste(imagen, factor_contraste=1.2):
    # Multiplicar los valores de los píxeles por el factor de contraste
    imagen_contraste = cv2.convertScaleAbs(imagen, alpha=factor_contraste, beta=0)
    return imagen_contraste

# Función para escalar la imagen
def escalar_imagen(imagen, factor_escala=1.2):
    return cv2.resize(imagen, None, fx=factor_escala, fy=factor_escala, interpolation=cv2.INTER_LINEAR)

# Funciones para las transformaciones
def rotar_imagen(imagen, angulo):
    (alto, ancho) = imagen.shape[:2]
    centro = (ancho // 2, alto // 2)
    matriz_rotacion = cv2.getRotationMatrix2D(centro, angulo, 1.0)
    return cv2.warpAffine(imagen, matriz_rotacion, (ancho, alto))

def ajustar_brillo(imagen, valor_brillo=30):
    hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v, valor_brillo)
    v[v > 255] = 255
    v[v < 0] = 0
    imagen_brillante = cv2.merge((h, s, v))
    return cv2.cvtColor(imagen_brillante, cv2.COLOR_HSV2BGR)

def ajustar_contraste(imagen, factor_contraste=1.2):
    return cv2.convertScaleAbs(imagen, alpha=factor_contraste, beta=0)

def aplicar_gaussian_blur(imagen, kernel_size=(5,5)):
    return cv2.GaussianBlur(imagen, kernel_size, 0)

def agregar_ruido_gaussiano(imagen, mean=0, var=0.1):
    row, col, ch = imagen.shape
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    return np.clip(imagen + gauss.reshape(row, col, ch), 0, 255).astype(np.uint8)

def transformar_perspectiva(imagen):
    altura, ancho = imagen.shape[:2]
    pts1 = np.float32([[0,0], [ancho,0], [0,altura], [ancho,altura]])
    pts2 = np.float32([[50,50], [ancho-50,50], [50, altura-50], [ancho-50, altura-50]])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    return cv2.warpPerspective(imagen, M, (ancho, altura))

# Rotación con un rango de ángulos completo
def rotar_imagen_extremo(imagen):
    angulo = random.uniform(0, 360)  # Rotación extrema en cualquier ángulo
    (alto, ancho) = imagen.shape[:2]
    centro = (ancho // 2, alto // 2)
    matriz_rotacion = cv2.getRotationMatrix2D(centro, angulo, 1.0)
    return cv2.warpAffine(imagen, matriz_rotacion, (ancho, alto))

# Ajuste de brillo extremo
def ajustar_brillo_extremo(imagen, valor_brillo=100):
    hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v, valor_brillo)
    v[v > 255] = 255
    v[v < 0] = 0
    imagen_brillante = cv2.merge((h, s, v))
    return cv2.cvtColor(imagen_brillante, cv2.COLOR_HSV2BGR)

# Ajuste de contraste extremo
def ajustar_contraste_extremo(imagen, factor_contraste=2.0):
    return cv2.convertScaleAbs(imagen, alpha=factor_contraste, beta=0)

# Desenfoque gaussiano extremo
def aplicar_gaussian_blur_extremo(imagen, kernel_size=(15, 15)):
    return cv2.GaussianBlur(imagen, kernel_size, 0)

# Ruido gaussiano extremo
def agregar_ruido_gaussiano_extremo(imagen, mean=0, var=0.5):
    row, col, ch = imagen.shape
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    return np.clip(imagen + gauss.reshape(row, col, ch), 0, 255).astype(np.uint8)

# Transformación de perspectiva extrema
def transformar_perspectiva_extremo(imagen):
    altura, ancho = imagen.shape[:2]
    margen = int(min(ancho, altura) * 0.3)  # Aumentar la distorsión
    pts1 = np.float32([[0, 0], [ancho, 0], [0, altura], [ancho, altura]])
    pts2 = np.float32([[margen, margen], [ancho - margen, margen], [margen, altura - margen], [ancho - margen, altura - margen]])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    return cv2.warpPerspective(imagen, M, (ancho, altura))

### Aplicación de Transformaciones

A continuación, iteramos sobre las imágenes en cada uno de los directorios especificados (`train/images`, `valid/images` y `test/images`).

> Para cada imagen:
1. **Rotaciones**: Rotamos la imagen en varios ángulos.
2. **Volteo**: Volteamos la imagen tanto horizontal como verticalmente.
3. **Ajuste de Brillo y Contraste**: Aplicamos varias modificaciones al brillo y contraste de la imagen, generando diferentes versiones.
4. **Escalado**: Aumentamos el tamaño de la imagen en dos factores (1.5x y 2.5x).

Las imágenes resultantes se guardan en un directorio temporal de salida.


In [ ]:
# Lista de directorios a procesar
directorios = [('data/train/images', 'data/train/imagesOPENCV'),
              ('data/valid/images', 'data/valid/imagesOPENCV'),
              ('data/test/images', 'data/test/imagesOPENCV')]

# Procesar cada directorio
for directorio_imagenes, directorio_modificado in directorios:
    # Crear la carpeta de salida si no existe
    os.makedirs(directorio_modificado, exist_ok=True)

    # Procesar todas las imágenes del directorio
    for archivo_imagen in os.listdir(directorio_imagenes):
        img_path = os.path.join(directorio_imagenes, archivo_imagen)
        imagen = cv2.imread(img_path)
        if imagen is None:
            print(f"No se pudo cargar la imagen {archivo_imagen}.")
            continue
        print(f"Procesando imagen: {archivo_imagen}")

        # Rotaciones
        for angulo in [35, 65, 90, 180, 270]:
            imagen_rotada = rotar_imagen(imagen, angulo)
            output_path = os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_rot{angulo}.jpg")
            cv2.imwrite(output_path, imagen_rotada)

        # Volteo
        imagen_volteada_h = cv2.flip(imagen, 1)  # Volteo horizontal
        imagen_volteada_v = cv2.flip(imagen, 0)  # Volteo vertical
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_flipH.jpg"), imagen_volteada_h)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_flipV.jpg"), imagen_volteada_v)

        # Ajuste de brillo
        imagen_brillo_aumentado = ajustar_brillo(imagen, 85)
        imagen_brillo_aumentado_2 = ajustar_brillo(imagen, 50)
        imagen_brillo_reducido = ajustar_brillo(imagen, -50)
        imagen_brillo_reducido_2 = ajustar_brillo(imagen, -25)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_brilloAumentado.jpg"), imagen_brillo_aumentado)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_brilloAumentado_2.jpg"), imagen_brillo_aumentado_2)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_brilloReducido.jpg"), imagen_brillo_reducido)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_brilloReducido_2.jpg"), imagen_brillo_reducido_2)

        # Ajuste de contraste
        imagen_contraste_aumentado = ajustar_contraste(imagen, 1.9)
        imagen_contraste_aumentado_2 = ajustar_contraste(imagen, 1.35)
        imagen_contraste_reducido = ajustar_contraste(imagen, 0.7)
        imagen_contraste_reducido_2 = ajustar_contraste(imagen, 0.45)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_contrasteAumentado.jpg"), imagen_contraste_aumentado)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_contrasteAumentado_2.jpg"), imagen_contraste_aumentado_2)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_contrasteReducido.jpg"), imagen_contraste_reducido)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_contrasteReducido_2.jpg"), imagen_contraste_reducido_2)

        # Escalado
        imagen_escalada = escalar_imagen(imagen, 1.5)
        imagen_escalada_2 = escalar_imagen(imagen, 2.5)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_escalado.jpg"), imagen_escalada)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_escalado_2.jpg"), imagen_escalada_2)

        # Aplicar transformaciones extremas
        imagen_rotada_extremo = rotar_imagen_extremo(imagen)
        imagen_brillo_extremo = ajustar_brillo_extremo(imagen, 100)  # Aumentar brillo en 100
        imagen_contraste_extremo = ajustar_contraste_extremo(imagen, 2.0)  # Duplicar el contraste
        imagen_blur_extremo = aplicar_gaussian_blur_extremo(imagen)  # Desenfoque extremo con un kernel mayor
        imagen_ruido_extremo = agregar_ruido_gaussiano_extremo(imagen)  # Ruido extremo con alta varianza
        imagen_perspectiva_extremo = transformar_perspectiva_extremo(imagen)  # Perspectiva extrema
        imagen_rotada = rotar_imagen(imagen, random.uniform(0, 360))
        imagen_blur = aplicar_gaussian_blur(imagen)
        imagen_ruido = agregar_ruido_gaussiano(imagen)
        imagen_perspectiva = transformar_perspectiva(imagen)

        # Guardar imágenes modificadas
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_rotada_extrema.jpg"), imagen_rotada_extremo)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_brillo_extremo.jpg"), imagen_brillo_extremo)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_contraste_extremo.jpg"), imagen_contraste_extremo)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_blur_extremo.jpg"), imagen_blur_extremo)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_ruido_extremo.jpg"), imagen_ruido_extremo)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_perspectiva_extrema.jpg"), imagen_perspectiva_extremo)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_rotada.jpg"), imagen_rotada)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_blur.jpg"), imagen_blur)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_ruido.jpg"), imagen_ruido)
        cv2.imwrite(os.path.join(directorio_modificado, f"{Path(archivo_imagen).stem}_perspectiva.jpg"), imagen_perspectiva)
        print("YOLO! (¡Guardada con éxito!)")

print("EXITO.")

> ### Definición de la función mover_imagenes:
*   `carpeta_origen`: La ruta de la carpeta donde están las imágenes que queremos mover.
*   `carpeta_destino`: La ruta a la que queremos mover las imágenes.



In [ ]:
# Función para mover las imágenes de una carpeta a otra
def mover_imagenes(carpeta_origen, carpeta_destino):
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)  # Crear la carpeta de destino si no existe
    for archivo in os.listdir(carpeta_origen):
        origen = os.path.join(carpeta_origen, archivo)
        destino = os.path.join(carpeta_destino, archivo)
        if os.path.isfile(origen):
            shutil.move(origen, destino)

# Mover las imágenes de las diferentes carpetas
mover_imagenes('data/train/imagesOPENCV', 'data/train/images')
mover_imagenes('data/valid/imagesOPENCV', 'data/valid/images')
mover_imagenes('data/test/imagesOPENCV', 'data/test/images')
print("Imágenes movidas exitosamente.")

# **# Autolabels**
## Generación de Pseudo-Labels utilizando YOLOv

Este notebook realiza inferencias utilizando un modelo YOLO previamente entrenado y guarda las predicciones como pseudo-labels en formato YOLO.

### Movimiento de las imagenes
Si no se encuentran predicciones (es decir, etiquetas) en la imagen, esta se mueva a una carpeta específica (data/train/imagesSINLABELS) junto con su archivo .txt correspondiente praa verificar.


In [ ]:
# Cargar tu propio modelo YOLO
model = YOLO('models/cs2_cv-woacm-noparams-yolo10m-1200epoch.pt')  # Reemplaza con tu modelo

# Ruta a las imágenes no etiquetadas:
directorio_imagenes = 'data/train/images'
# Directorio de salida para imágenes con nuevos labels
directorio_salida = 'data/train/imagesNUEVOSLABELS'
os.makedirs(directorio_salida, exist_ok=True)
# Directorio de salida para imágenes sin labels
directorio_sin_labels = 'data/train/imagesSINLABELS'
os.makedirs(directorio_sin_labels, exist_ok=True)
# Directorio de salida para etiquetas vacías
directorio_labels_sin_labels = 'data/test/labelsSINLABELS'
os.makedirs(directorio_labels_sin_labels, exist_ok=True)

# Función para guardar las pseudo-labels en formato YOLO
def guardar_labels_yolo(imagen, predicciones, directorio_salida):
    nombre_imagen = Path(imagen).stem
    with open(f"{directorio_salida}/{nombre_imagen}.txt", 'w') as f:
        for x1, y1, x2, y2, conf, clase in predicciones:
            if conf > 0.7:  # Solo guardar predicciones con alta confianza
                # Convertir las coordenadas a formato YOLO (normalizado)
                x_centro = (x1 + x2) / 2.0
                y_centro = (y1 + y2) / 2.0
                ancho = x2 - x1
                alto = y2 - y1

                # Normalizar usando el tamaño de la imagen
                img = cv2.imread(imagen)
                img_width = img.shape[1]
                img_height = img.shape[0]
                x_centro /= img_width
                y_centro /= img_height
                ancho /= img_width
                alto /= img_height

                # Guardar en formato YOLO .txt
                f.write(f"{int(clase)} {x_centro} {y_centro} {ancho} {alto}\n")

# Realizar inferencia en todas las imágenes no etiquetadas
for archivo_imagen in os.listdir(directorio_imagenes):
    img_path = os.path.join(directorio_imagenes, archivo_imagen)
    img = cv2.imread(img_path)

    # Realizar inferencia con el modelo YOLOvX
    resultados = model(img)

    # Extraer los valores de las cajas, confianza y clases
    boxes = resultados[0].boxes  # Obtenemos las cajas de predicción
    predicciones = []

    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # Coordenadas de la caja
        conf = box.conf.cpu().numpy()[0]  # Confianza de la predicción
        clase = box.cls.cpu().numpy()[0]  # Clase predicha
        predicciones.append([x1, y1, x2, y2, conf, clase])

    if len(predicciones) > 0:  # Si hay predicciones, guardamos las labels
        guardar_labels_yolo(img_path, predicciones, directorio_salida)
    else:  # Si no hay predicciones, movemos la imagen a la carpeta de imágenes sin labels
        shutil.move(img_path, os.path.join(directorio_sin_labels, archivo_imagen))

        # Crear un archivo .txt vacío y moverlo a la carpeta de labels sin labels
        nombre_imagen = Path(img_path).stem
        empty_label_path = os.path.join(directorio_labels_sin_labels, f"{nombre_imagen}.txt")
        open(empty_label_path, 'w').close()

print(f"Proceso completado. Imágenes sin labels movidas a {directorio_sin_labels}. Labels generados y guardados en {directorio_salida}.")

# **# LabelsImg**:
> CRASHEA

El plan es que le demos el directorio de donde se encuentran las imagenes y labels que no han encontrado etiquetas y obligar a abrir con el programa labelImg para poder etiquetar manualmente todas las imagenes

In [ ]:
# Directorio que contiene tanto las imágenes como las etiquetas
directorio = 'data/test/imagesSINLABELS'  # Cambia esto según tu ruta
directorio_labels = 'data/test/labelsSINLABELS2'  # Cambia esto según tu ruta

# Procesar todas las imágenes y etiquetas
for archivo_imagen in os.listdir(os.path.join(directorio,)):
    img_path = os.path.join(directorio, archivo_imagen)
    label_path = os.path.join(directorio_labels, Path(archivo_imagen).stem + '.txt')

    # Comprobar si el archivo de etiqueta está vacío
    if not os.path.exists(label_path) or os.stat(label_path).st_size == 0:
        print(f"Abrir LabelImg para etiquetar {archivo_imagen}")

        # Abrir LabelImg para esa imagen
        if os.name == 'nt':  # Windows
            subprocess.run(['labelImg', img_path])
        else:  # Linux o macOS
            subprocess.run(['python', 'labelImg.py', img_path])

        # Esperar hasta que el usuario termine de etiquetar antes de continuar
        input("Presiona Enter cuando hayas terminado de etiquetar la imagen en LabelImg...")
    else:
        print(f"Etiqueta ya existente para {archivo_imagen}.")


# **# Checklabels**

### Visualización y verificación de las imagenes

In [ ]:
# Directorio de imágenes y etiquetas
directorio_imagenes = ('data/train/images')  # Carpeta de imágenes
directorio_labels = ('data/train/labels')  # Carpeta de etiquetas

# Diccionario para las clases (ajústalo según tus clases)
clases = {0: 'ct', 1: 'cthead', 2: 't', 3: 'thead'}  # Ajusta esto según tus clases

def visualizar_labels_en_imagen(imagen_path, label_path):
    img = cv2.imread(imagen_path)
    img_height, img_width = img.shape[:2]

    # Leer el archivo de etiquetas
    with open(label_path, 'r') as f:
        labels = f.readlines()

    # Dibujar cada etiqueta en la imagen
    for label in labels:
        clase_id, x_centro, y_centro, ancho, alto = map(float, label.split())

        # Desnormalizar las coordenadas y dimensiones (convertir a píxeles)
        x_centro *= img_width
        y_centro *= img_height
        ancho *= img_width
        alto *= img_height

        # Obtener las esquinas del bounding box
        x1 = int(x_centro - ancho / 2)
        y1 = int(y_centro - alto / 2)
        x2 = int(x_centro + ancho / 2)
        y2 = int(y_centro + alto / 2)

        # Dibujar el bounding box en la imagen
        color = (0, 255, 0)  # Verde
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

        # Añadir el nombre de la clase
        text = clases[int(clase_id)]
        cv2.putText(img, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Mostrar la imagen con los bounding boxes
    cv2.imshow("Imagen con Labels", img)
    cv2.waitKey(0)  # Presiona cualquier tecla para cerrar la ventana
    cv2.destroyAllWindows()

# **# TOOLS**


*   **Autofiltering:** Codigo para separar imagenes y labels sin etiquetado de las ya clasificadas en sus clases.
*   More...



In [ ]:
# Directorio de imágenes y etiquetas
directorio_imagenes = 'data/valid/nolabels'  # Carpeta de imágenes
directorio_labels = 'data/valid/nolabels2'  # Carpeta de etiquetas
directorio_sin_label = 'data/valid/nolabelsinlabel/'  # Carpeta para imágenes y etiquetas sin labels

# Crear la carpeta 'nolabels' si no existe
os.makedirs(directorio_sin_label, exist_ok=True)

# Procesar todas las imágenes y etiquetas
for archivo_imagen in os.listdir(directorio_imagenes):
    img_path = os.path.join(directorio_imagenes, archivo_imagen)
    label_path = os.path.join(directorio_labels, Path(archivo_imagen).stem + '.txt')

    # Verificar si el archivo de etiqueta existe
    if os.path.exists(label_path):
        # Leer el archivo de etiqueta
        with open(label_path, 'r') as f:
            labels = f.readlines()

        # Si el archivo de etiqueta está vacío o no contiene etiquetas, mover imagen y label
        if len(labels) == 0:
            print(f"Moviéndo {archivo_imagen} y su label correspondiente a la carpeta 'nolabels'")
            shutil.move(img_path, os.path.join(directorio_sin_label, archivo_imagen))
            shutil.move(label_path, os.path.join(directorio_sin_label, Path(label_path).name))
    else:
        # Si no existe el archivo de etiqueta, mover solo la imagen
        print(f"No se encontró el label para {archivo_imagen}, moviendo a la carpeta 'nolabels'")
        shutil.move(img_path, os.path.join(directorio_sin_label, archivo_imagen))

print("Proceso completo. Las imágenes y etiquetas sin labels han sido movidas a la carpeta 'nolabels'.")